In [32]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue
import pyaudio
import subprocess
import json
#from vosk import Model, KaldiRecognizer
from deepspeech import Model
import time
import numpy as np
import whisper
import torch

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECS = 20
SAMPLE_SIZE = 2
BEAM_WIDTH = 100
ALPHA = 0.93
BETA = 1.18
FINAL_STR = []

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description="Record",
    disabled = False,
    button_style = "success",
    icon = 'microphone'
)

stop_button = widgets.Button(
    description="Stop",
    disabled = False,
    button_style = "warning",
    icon = 'stop'
)

output = widgets.Output()

def record_mic(chunk = CHUNK):
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels = CHANNELS, rate = RATE, input = True, frames_per_buffer=chunk)

    frames = []
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
 
        if len(frames) >= RATE//CHUNK*RECORD_SECS:
            recordings.put(frames.copy())
            frames = []
        
    stream.stop_stream()
    stream.close()
    p.terminate()

'''
model = Model('/Users/connia/vosk-model-en-us-0.22')
rec = KaldiRecognizer(model,RATE)
rec.SetWords(True)
'''

'''
model = Model('./models/deepspeech-0.9.3-models.pbmm')
model.enableExternalScorer('./models/deepspeech-0.9.3-models.scorer')
model.setBeamWidth(BEAM_WIDTH)
model.setScorerAlphaBeta(ALPHA,BETA)
stream_model = model.createStream()
'''

model = whisper.load_model('base.en')


In [33]:
def speech_recognition(output):
    while not messages.empty():
        frames = recordings.get()

        #rec.AcceptWaveform(b''.join(frames))
        #result = rec.Result()
        data16 = np.frombuffer(b''.join(frames), dtype=np.int16)
        data32 = data16.astype(np.float32)/32768.0
        
        #stream_model.feedAudioContent(data16)
        #text = json.loads(result)['text']
        result = model.transcribe(data32)
        FINAL_STR.append(result["text"]+'\n')

        #case = subprocess.check_output("python vosk-recasepunc-en-0.22/recasepunc.py predict vosk-recasepunc-en-0.22/checkpoint", shell = True, text = True, input = text)
        #print(stream_model.intermediateDecode())
        output.append_stdout(result["text"]+'\n')
        #time.sleep(1)
    


def start_recording(data):
    messages.put(True)
    with output:
        display("Starting recording ...")
        record = Thread(target = record_mic)
        record.start()

        transcribe = Thread(target = speech_recognition, args= (output, ))
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        display ("Recording Finished. ")
        #FINAL_STR = stream_model.finishStream()
        display(''.join(FINAL_STR))

In [34]:
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)
print(FINAL_STR)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

/Users/connia/tmp/deepspeech-venv/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
